In [1]:
import astropy.io.fits as fits
import numpy as np

# Convert the Windows path to WSL2 path format:
# "G:\data\PhD Projects\SR\sim_datamaps\SHARK_1_SPIRE250_smoothed_Jy_beam.fits"
# becomes:
file_path = '/mnt/g/data/PhD Projects/SR/sim_datamaps/SHARK_1_SPIRE250_smoothed_Jy_beam.fits'

# Open the FITS file
hdul = fits.open(file_path)

hdr = hdul[0].header
hdr

SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                  -64 / array data type                                
NAXIS   =                    2 / number of array dimensions                     
NAXIS1  =                 1199                                                  
NAXIS2  =                  601                                                  
WCSAXES =                    2 / Number of coordinate axes                      
CRPIX1  =      599.89398946613 / Pixel coordinate of reference point            
CRPIX2  =      301.35606954204 / Pixel coordinate of reference point            
CDELT1  =   0.0016666666666667 / [deg] Coordinate increment at reference point  
CDELT2  =   0.0016666666666667 / [deg] Coordinate increment at reference point  
CUNIT1  = 'deg'                / Units of coordinate increment and value        
CUNIT2  = 'deg'                / Units of coordinate increment and value        
CTYPE1  = 'RA---TAN'        

In [2]:
from astropy.wcs import WCS

def pixel_to_radec(wcs_obj, x, y, origin=0):
    """
    Converts pixel coordinates (x, y) to world coordinates (RA, Dec) using a WCS object.

    Parameters:
        wcs_obj (WCS): The WCS object.
        x (int, float, or array-like): Pixel coordinate(s) along the x-axis.
        y (int, float, or array-like): Pixel coordinate(s) along the y-axis.
        origin (int): Pixel coordinate origin; use 0 for Python indexing.
        
    Returns:
        ra, dec: The corresponding right ascension and declination.
    """
    world_coords = wcs_obj.all_pix2world(x, y, origin)
    # In case x, y are arrays, world_coords will be an array of shape (N, 2)
    if hasattr(world_coords, "shape") and world_coords.shape[-1] == 2:
        ra, dec = world_coords[..., 0], world_coords[..., 1]
    else:
        ra, dec = world_coords[0], world_coords[1]
    return ra, dec

In [3]:
def interp_header_1(original_header, interp_pixscale):
    # Copy, compute scale
    new_header = original_header.copy()
    orig_scale = abs(original_header["CDELT1"] * 3600)  # arcsec/pix
    scaler    = orig_scale / interp_pixscale

    # Let astropy do the slicing for CRPIX & CDELT
    w0     = WCS(original_header)
    slice_hdr = w0[::1/scaler, ::1/scaler].to_header()

    # Pick off only the WCS keywords we care about
    for key in ["CRPIX1", "CRPIX2", "CDELT1", "CDELT2"]:
        new_header[key] = slice_hdr[key]

    # Update the image size
    new_header["NAXIS1"] = int(np.round(original_header["NAXIS1"] * scaler + 0.5))
    new_header["NAXIS2"] = int(np.round(original_header["NAXIS2"] * scaler + 0.5))

    return WCS(new_header)

def interp_header_2(original_header, interp_pixscale):
    # Copy header and compute scaler
    new_header = original_header.copy()
    orig_scale = abs(original_header["CDELT1"] * 3600)  # arcsec/pix
    scaler    = orig_scale / interp_pixscale

    # Manually compute new CRPIX
    new_crpix1 = (original_header["CRPIX1"] - 1) * scaler + 1
    new_crpix2 = (original_header["CRPIX2"] - 1) * scaler + 1

    # Update only the WCS keys
    new_header["CRPIX1"] = new_crpix1
    new_header["CRPIX2"] = new_crpix2
    new_header["CDELT1"] = np.sign(original_header["CDELT1"]) * interp_pixscale/3600
    new_header["CDELT2"] = np.sign(original_header["CDELT2"]) * interp_pixscale/3600

    # Update image size
    new_header["NAXIS1"] = int(np.round(original_header["NAXIS1"] * scaler + 0.5))
    new_header["NAXIS2"] = int(np.round(original_header["NAXIS2"] * scaler + 0.5))

    # And your final WCS
    interp_wcs = WCS(new_header)
    return interp_wcs

def interp_header_3(original_header, interp_pixscale):
    # (your function, slightly simplified to return only w_interp)
    wcs_orig = WCS(original_header)
    scale_factor = abs(original_header["CDELT1"]*3600) / interp_pixscale

    # Compute new dims
    naxis1 = int(np.round(original_header["NAXIS1"] * scale_factor + 0.5))
    naxis2 = int(np.round(original_header["NAXIS2"] * scale_factor + 0.5))

    # Build the interpolated WCS
    w_interp = WCS(naxis=2)
    w_interp.wcs.crval = [original_header["CRVAL1"], original_header["CRVAL2"]]
    w_interp.wcs.crpix = [original_header["CRPIX1"] * scale_factor,
                          original_header["CRPIX2"] * scale_factor]
    w_interp.wcs.cdelt = [np.sign(original_header["CDELT1"]) * interp_pixscale/3600,
                          np.sign(original_header["CDELT2"]) * interp_pixscale/3600]
    w_interp.wcs.ctype = ["RA---TAN", "DEC--TAN"]
    w_interp.wcs.cunit = ["deg", "deg"]

    # (we skip the actual reproject here, since we’re only testing WCS)
    return w_interp

def interp_header_4(original_header, interp_pixscale):
    # (your function, slightly simplified to return only w_interp)
    wcs_orig = WCS(original_header)
    scale_factor = abs(original_header["CDELT1"]*3600) / interp_pixscale

    # Compute new dims
    naxis1 = int(np.round(original_header["NAXIS1"] * scale_factor + 0.5))
    naxis2 = int(np.round(original_header["NAXIS2"] * scale_factor + 0.5))

    # Manually compute new CRPIX
    new_crpix1 = (original_header["CRPIX1"] - 1) * scale_factor + 1
    new_crpix2 = (original_header["CRPIX2"] - 1) * scale_factor + 1

    # Build the interpolated WCS
    w_interp = WCS(naxis=2)
    w_interp.wcs.crval = [original_header["CRVAL1"], original_header["CRVAL2"]]
    w_interp.wcs.crpix = [new_crpix1, new_crpix2]
    w_interp.wcs.cdelt = [np.sign(original_header["CDELT1"]) * interp_pixscale/3600,
                          np.sign(original_header["CDELT2"]) * interp_pixscale/3600]
    w_interp.wcs.ctype = ["RA---TAN", "DEC--TAN"]
    w_interp.wcs.cunit = ["deg", "deg"]

    # (we skip the actual reproject here, since we’re only testing WCS)
    return w_interp

In [4]:
interp_hdr_1 = interp_header_1(hdr, 1)
interp_hdr_2 = interp_header_2(hdr, 1)
interp_hdr_3 = interp_header_3(hdr, 1)
interp_hdr_4 = interp_header_4(hdr, 1)

wcs_original = WCS(hdr)

# Test which interpolation method is correct
x_orig, y_orig = 2, 0
xnew, ynew = 2*6, 0

ra1, dec1 = pixel_to_radec(wcs_original, x_orig, y_orig)
ra2, dec2 = pixel_to_radec(interp_hdr_1, xnew, ynew)
ra3, dec3 = pixel_to_radec(interp_hdr_2, xnew, ynew)
ra4, dec4 = pixel_to_radec(interp_hdr_3, xnew, ynew)
ra5, dec5 = pixel_to_radec(interp_hdr_4, xnew, ynew)

print(f"Original RA, Dec: {ra1}, {dec1}")
print(f"Interpolated RA, Dec (method 1): {ra2}, {dec2}")
print(f"Interpolated RA, Dec (method 2): {ra3}, {dec3}")
print(f"Interpolated RA, Dec (method 3): {ra4}, {dec4}")
print(f"Interpolated RA, Dec (method 4): {ra5}, {dec5}")

Original RA, Dec: 211.50223102373332, -4.49990123948753
Interpolated RA, Dec (method 1): 211.50153382274559, -4.500594576657694
Interpolated RA, Dec (method 2): 211.50223102373334, -4.499901239487518
Interpolated RA, Dec (method 3): 211.50083662087107, -4.5012879127252585
Interpolated RA, Dec (method 4): 211.50223102373332, -4.49990123948753


In [5]:
# Another test using different pixel coordinates
x_orig2, y_orig2 = 100, 100
scaling_factor = 6  # because original CDELT is 6 arcsec/pix and interp_pixscale is 1 arcsec/pixel.
xnew2, ynew2 = x_orig2 * scaling_factor, y_orig2 * scaling_factor

ra_orig2, dec_orig2 = pixel_to_radec(wcs_original, x_orig2, y_orig2)
ra_interp1, dec_interp1 = pixel_to_radec(interp_hdr_1, xnew2, ynew2)
ra_interp2, dec_interp2 = pixel_to_radec(interp_hdr_2, xnew2, ynew2)
ra_interp3, dec_interp3 = pixel_to_radec(interp_hdr_3, xnew2, ynew2)

print("Test 2:")
print(f"Original RA, Dec: {ra_orig2}, {dec_orig2}")
print(f"Interpolated RA, Dec (method 1): {ra_interp1}, {dec_interp1}")
print(f"Interpolated RA, Dec (method 2): {ra_interp2}, {dec_interp2}")
print(f"Interpolated RA, Dec (method 3): {ra_interp3}, {dec_interp3}")

Test 2:
Original RA, Dec: 211.6661910063106, -4.333465963455973
Interpolated RA, Dec (method 1): 211.66549402242475, -4.334159546859495
Interpolated RA, Dec (method 2): 211.6661910063106, -4.333465963455975
Interpolated RA, Dec (method 3): 211.66479703760402, -4.334853129281773
